In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 9
### Team Member Names: Vivan Garg, Brian Liu, Bogdan Basaraba
### Team Strategy Chosen: RISKY

In [2]:
# Reads the provided Tickers.csv file, and outputs a dataframe of all tickers. 
tickers = pd.read_csv("Tickers.csv")

In [3]:
# Creating an empty list, where the ticker names will be stored. 
tickers_lst = []

# A for loop to append all tickers from the DataFrame into a list
for i in range(len(tickers)):
    tickers_lst.append(tickers.iloc[i][0])

# One last ticker remaining in the column header, so also append that to the list 
tickers_lst.append(tickers.columns.values[0])

In [4]:
# Create an empty list where valid tickers will be stored 
valid_tickers = []
# For loop to check if each ticker is valid according to instructions, if true then append to the list 
for each_ticker in tickers_lst:
    stock = yf.Ticker(each_ticker)
    avg_volume= stock.history(start="2021-07-02", end = "2021-10-21").Volume.mean()
    if avg_volume >= 10000 and stock.info['market'] == 'us_market':
        valid_tickers.append(each_ticker)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


In [5]:
# Variables for start dates and end dates to get the history of the valid tickers 
start_date='2018-11-26' 
end_date="2021-11-26"

In [6]:
# Creates a dictionary with the tickers as keys and their closing prices as values
ticker_hist_dict = {}
for each_ticker in valid_tickers:
    ticker = yf.Ticker(each_ticker)
    close = ticker.history(start=start_date, end=end_date).Close
    ticker_hist_dict.update({each_ticker: close})

In [7]:
# Creating an empty list that will store the tickers of our final portfolio 
portfolio = []

In [8]:
# Adding the the stock with maximum standard deviation as our first stock in the portfolio
ticker_hist_df = pd.DataFrame(ticker_hist_dict)
std = pd.DataFrame(ticker_hist_df.pct_change().std())
std.columns = ["Standard deviations"]
max_std = max(ticker_hist_df.pct_change().std())
for index, row in std.iterrows():
    if max_std == row["Standard deviations"]:
        portfolio.append(index)

In [9]:
# Creating a dictionary that will store the correlation between all the remaining stocks
# and the stock with the maximum standard deviation in the portfolio that was previously chosen
ticker = yf.Ticker(portfolio[0]).history(start=start_date, end =end_date).Close
correlation = {}
ticker_df = pd.DataFrame(ticker)
for each_ticker in valid_tickers: 
    if each_ticker != portfolio[0]:
        ticker_df[each_ticker] = yf.Ticker(each_ticker).history(start=start_date, end =end_date).Close      
        correlation.update({each_ticker:ticker_df.Close.corr(ticker_df[each_ticker])})
correlation

{'ABBV': -0.4755277386906867,
 'ABT': -0.5884065640414882,
 'ACN': -0.427840564116365,
 'AIG': 0.447006293715876,
 'AMZN': -0.6873502094880347,
 'AXP': 0.007133398109575831,
 'BA': 0.906607246319593,
 'BAC': 0.011868726562560359,
 'BIIB': -0.06768867657835459,
 'BK': 0.4559184335323583,
 'BLK': -0.4439089129684044,
 'BMY': -0.6660835719631801,
 'C': 0.3636777360288481,
 'CAT': -0.29823851813227437,
 'CL': -0.6399069370852489,
 'CMCSA': -0.32905670528129377,
 'COF': -0.015176683655408266,
 'COP': 0.7251015038112363,
 'COST': -0.5755893429107787,
 'CSCO': 0.27933050588346453,
 'CVS': -0.24809448178727495,
 'GM': 0.02222237172345517,
 'GOOG': -0.36942958589548414,
 'JPM': -0.10500190286463204,
 'KMI': 0.5723107468270291,
 'KO': -0.18565509694572557,
 'LLY': -0.4240704928632022,
 'LMT': -0.5735506942135117,
 'MO': 0.30422474655917925,
 'MON': 0.14041613615640425,
 'MRK': -0.2074730673645045,
 'MS': -0.2756506880606098,
 'MSFT': -0.5891313460677001,
 'NEE': -0.6672960785796046,
 'NKE': -0.4

In [10]:
# Choosing the remaining 9 stocks according to their correlation with the first stock
for i in range(9):
    portfolio.append(max(correlation, key=correlation.get))
    del(correlation[max(correlation, key=correlation.get)])

In [11]:
# printing out our final portfolio 
portfolio

['OXY', 'SLB', 'BA', 'SPG', 'COP', 'KMI', 'BK', 'AIG', 'USB', 'C']

In [12]:
# Defining function that returns a dataframe with tickers and their closing prices
def history(list_of_tickers, start_date, end_date):
    hist_dict = {} 
    for ticker in list_of_tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, interval="1mo").Close
        hist_dict.update({ticker: hist})

    return pd.DataFrame(hist_dict).dropna()

In [13]:
# Defining function that caclulates number of shares based on list_of_pcts that contains the weighting of each stock in list_of_tickers
def num_shares_pct(list_of_tickers, hist_df, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        stock_price = hist_df.loc[:, ticker][0]
        num_shares.append((100000*(list_of_pcts[i]/100))/(stock_price))
        i += 1
    return num_shares

In [14]:
# The weighting of each stock in our portfolio
percentages = [35, 25, 5, 5, 5, 5, 5, 5, 5, 5]

In [15]:
# extracting history of the stocks in our portolio
hist_df = history(portfolio, start_date, end_date)

In [16]:
# The date we will buy our portfolio on
buy_date = "2021-11-26"

# Creating a list with the buying price of all the stocks in our portfolio
buy_price = []
for each_ticker in portfolio:
    closing_price = yf.Ticker(each_ticker).history(start=buy_date).Close.values[0]
    buy_price.append(closing_price)

In [17]:
# The number of shares bought for each stock based on our start_date 
shares = num_shares_pct(portfolio, hist_df, percentages)

In [18]:
# Defining function that calculates the portfolio based on the num of shares of each ticker in the list_of_tickers 
def portfolio_calc(list_of_tickers, hist_df, num_shares):
    value = {}
    n = 0
    for ticker in list_of_tickers:
        value.update({ticker: hist_df[ticker]*num_shares[n]})
        n += 1

    each_stock_value = pd.DataFrame(value)
    return pd.DataFrame({"portfolio": each_stock_value.sum(axis=1)})

# Creating a dataframe with the portfolio value
portfolio_df = portfolio_calc(portfolio, hist_df, shares)

# Joining history of each ticker in portfolio with portfolio_df
hist_w_portfolio= pd.concat([hist_df, portfolio_df], join='inner', axis=1)

In [19]:
hist_w_portfolio

,OXY,SLB,BA,SPG,COP,KMI,BK,AIG,USB,C,portfolio
Date,,,,,,,,,,,
2018-12-01,54.953987,32.220173,313.372223,144.172501,56.871059,12.815855,43.307858,35.857159,41.162567,47.215565,100000.000000
2019-01-01,60.487411,39.911430,374.705719,156.299149,61.741814,15.082377,48.138245,39.671215,46.456997,58.461685,115127.176963
2019-02-01,59.916782,39.776028,427.507721,155.475266,61.887764,16.144539,48.538490,39.643684,46.938274,58.026356,115954.548556
2019-03-01,59.962063,39.789024,372.486572,158.132492,61.151691,16.860767,46.642460,39.515213,43.760025,56.826843,114672.553072
2019-04-01,53.972187,38.976254,368.843903,150.746994,57.834801,16.742798,45.930290,43.975292,48.793537,64.571800,111546.379688
2019-05-01,45.621532,31.679621,333.608978,140.671188,54.023136,17.020546,39.712799,47.210808,45.938404,56.762909,97987.548020
2019-06-01,46.089024,36.291389,357.526581,140.273666,56.170712,17.813984,41.070988,49.253803,47.951641,64.371582,104221.780691
2019-07-01,47.839920,37.023029,335.103210,142.416077,54.402706,17.592161,43.647800,52.073673,52.673336,65.410271,106753.395830
2019-08-01,40.499989,30.038946,357.605164,130.773438,48.293167,17.503021,39.384918,48.399963,48.562695,59.150524,93876.288828


In [20]:
# Calculating Beta
MarketIndex_hist = history(["^GSPC"], start_date, end_date)
index_w_portfolio = pd.concat([portfolio_df,MarketIndex_hist], join='inner', axis=1)

percent_change = index_w_portfolio.pct_change()
percent_change.drop(index=percent_change.index[0], inplace=True)

MarketVar = percent_change["^GSPC"].var()

Beta = percent_change.cov()/MarketVar
Beta_value = Beta.iat[0, 1]

In [21]:
# Calculating sharpe ratio assuming the risk free rate of return is 0
sharpe_ratio = pd.DataFrame(portfolio_df.pct_change().mean()/portfolio_df.pct_change().std()).iloc[0,0]

In [22]:
# Defining function that caclulates number of shares to be bought for each stock based on the prices in list_of_prices
def num_shares_lst(list_of_tickers, list_of_prices, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        num_shares.append((100000*(list_of_pcts[i]/100))/(list_of_prices[i]))
        i += 1
    return num_shares

In [23]:
# Calculating the number of shares bought
buy_shares = num_shares_lst(portfolio, buy_price, percentages)

In [24]:
# Creating dictionary with FinalPortfolio data
FinalPortfolio_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                 "Ticker": portfolio,
                       "Price": buy_price,
                       "Shares": buy_shares,
                      }
# Creating Dataframe from FinalPortfolio_dict
FinalPortfolio = pd.DataFrame(FinalPortfolio_dict) 
FinalPortfolio.set_index("index", inplace=True)
FinalPortfolio['Value'] = FinalPortfolio['Price']*FinalPortfolio['Shares']
FinalPortfolio["Weight"] = percentages
print("The total value of the portfolio is: $"+ str(FinalPortfolio.Value.sum(axis=0)))

The total value of the portfolio is: $100000.0


In [25]:
print("The total weight of the portfolio is: "+ str(FinalPortfolio.Weight.sum(axis=0))+"%")

The total weight of the portfolio is: 100%


In [26]:
FinalPortfolio

,Ticker,Price,Shares,Value,Weight
index,,,,,
1,OXY,29.700001,1178.451148,35000.0,35
2,SLB,29.700001,841.750820,25000.0,25
3,BA,199.210007,25.099141,5000.0,5
4,SPG,160.479996,31.156531,5000.0,5
5,COP,71.480003,69.949633,5000.0,5
6,KMI,16.260000,307.503071,5000.0,5
7,BK,56.750000,88.105727,5000.0,5
8,AIG,56.049999,89.206067,5000.0,5
9,USB,57.500000,86.956522,5000.0,5


In [27]:
# Creating dictionary with Stocks data
Stocks_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               "Ticker": portfolio,
               "Shares": buy_shares,}  
# Creating Dataframe from Stocks_dict 
Stocks = pd.DataFrame(Stocks_dict)
Stocks.set_index("index", inplace=True)
Stocks

,Ticker,Shares
index,,
1,OXY,1178.451148
2,SLB,841.750820
3,BA,25.099141
4,SPG,31.156531
5,COP,69.949633
6,KMI,307.503071
7,BK,88.105727
8,AIG,89.206067
9,USB,86.956522


In [28]:
# Export final dataframe to a CSV file 
Stocks.to_csv('Stocks_Group_9.csv', index=False)

Our final portfolio will only contain 10 stocks. The more diversified a portfolio is, the more the risk is typically reduced. Since our group decided to create a risky portfolio, we chose the minimum required amount of stocks because then our portfolio will be less diverse, implying it'll be more risky. 

We are using the past 3 years of history for each stock, if available, for all calculation purposes. The reason for not choosing a longer period is because if the period is too long than the data becomes outdated and unreliable and may not follow the current trends. While for a shorter period of time, the data is too less to be reliable and to able to perform an analysis. Therefore, 3 years seems like an optimal period.

We first calculate the standard deviation of the daily percentage change of the closing prices of each stock. Then, the initial stock that's chosen is based on the highest standard deviation. As we know, a high standard deviation means high volatility. That means the stock price is unpredictable and can change dramatically over a short period of time in any direction, resulting in a very risky stock. 

Then the next 9 stocks are chosen in order of highest positive correlation to the first stock. The reason for doing this is because since all of our stocks are highly correlated, most of the stocks in the portfolio are likely to move in one direction. Choosing highly correlated stocks will increase the risk of a portfolio. 

Ideally, a portolio with only one stock that is highly volatile is the riskiest possible portfolio but since there is a restriction on the maximum weight of a stock possible in our portfolio, the first stock (the most volatile one) is given 35% (the maximum possible) of our portfolio and the second stock which is the one that is highest positively correlated to the first one is given 25% (the maximum possible considering the rest of the stocks in the portfolio have to be at least 5% each), and the rest of the 8 stocks in our portfolio are given 5% each so that our portfolio has as little diversity as possible. The 35% of the total weight is assigned to the stock with the highest volatility because that stock has the greatest chance to fluctuate in price. 

The resulting Beta of the portfolio should be greater than 1, implying that the portfolio is more volatile than the broader market. If two portfolios offer similar returns, the one with higher standard deviation will have a lower Sharpe ratio, that is the risker one will have a lower sharpe ratio, therefore the sharpe ratio should be small.

In [29]:
print(f"Our portfolio's Beta is {Beta_value} and Sharpe ratio is {sharpe_ratio} which is as expected.")

Our portfolio's Beta is 1.9734210084880957 and Sharpe ratio is 0.051754077907389125 which is as expected.


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Vivan Garg, Brian Liu, Bogdan Basaraba.